In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import sqlite3 as sql
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [ ]:
# Establish connection to the database
db_path = r"C:\Users\user\Desktop\3Signet Project2 Task\ABCPharmData.db"
conn = sql.connect(db_path)

In [ ]:
# Load data into a DataFrame
query = "SELECT * FROM ABCData"
df = pd.read_sql(query, conn)
df.head()

### Data Preprocessing and Validation

In [ ]:
df.info()

In [ ]:
#Converting Quantity and Sales column to Integer type

df['Quantity'] = df['Quantity'].astype(int)
df['Sales'] = df['Sales'].astype(int)
print(df.dtypes)

In [ ]:
# Converting the 'Month' column to numeric format
df['Month'] = pd.to_datetime(df['Month'], format='%B').dt.month

In [ ]:
print(sorted(df['Month'].unique()))

In [ ]:
#Creating an object cursor to execute sql queries
cursor = conn.cursor()

In [ ]:
#CHECKING FOR DUPLICATES IN INDEX
rowz= cursor.execute("SELECT* FROM ABCData GROUP BY ID HAVING COUNT(*) > 1").fetchall()
print(rowz)

In [ ]:
#Now we want to confirm there are no duplicate rows in data\n",
df_removed = df.copy()

In [ ]:
 df_removed.drop(columns=["ID"])

In [ ]:
# Check for duplicate rows
duplicates = df_removed.duplicated()

# Print the number of duplicate rows
print(f"Number of duplicate rows: {duplicates.sum()}")

# If there are duplicates, show them
if duplicates.any():
    print("Duplicate rows:")
    print(df_removed[duplicates])
else:
    print("No duplicate rows found.")


In [ ]:
%load_ext nb_black
# Checking for negative sales amounts\n",
invalid_sales = df[df["Sales"] < 0]
if not invalid_sales.empty:
    raise ValueError("Invalid sales amounts found.")

In [ ]:
# Checking how many rows have negative values in the Quantity or Sales column\n",
rowz = cursor.execute(
    "SELECT COUNT(*) AS NegativeCount FROM ABCData WHERE Quantity < 0 OR Sales < 0"
).fetchall()
print(rowz)

In [ ]:
# Replacing negative values with their absolute values
df["Quantity"] = df["Quantity"].abs()
df["Sales"] = df["Sales"].abs()
print(df[df["Quantity"] < 0])

In [ ]:
rowz = cursor.execute(
    "SELECT COUNT(*) AS NegativeCount FROM ABCData WHERE Quantity = 0 OR Sales = 0"
).fetchall()
print(rowz)

In [ ]:
# Remove rows where 'Quantity' or 'Sales' have zero values
df = df[(df['Quantity'] != 0) & (df['Sales'] != 0)]

# Verify the changes
print(f"Number of rows after removal: {len(df)}")


In [ ]:
df.head()

# Performing Geospatial and Channel Analysis

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
# Creating a geopandas Dataframe
geometry = [Point(xy) for xy in zip(df["Longitude"], df["Latitude"])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

In [ ]:
gdf.head(3)

In [ ]:
# Visualizing a plot of the geopandas dataframe
fig, ax = plt.subplots(1, figsize=(6, 12))
gdf.plot(ax=ax, column="Country", legend=True, legend_kwds={"loc": "center left"})
leg = ax.get_legend()
leg.set_bbox_to_anchor((1.04, 0.5))

* From this plot, we can see more concentration of sales from Germany 


In [ ]:
print(df[df["Sales"] < 0])

In [ ]:
# Calculate IQR bounds
Q1 = df["Quantity"].quantile(0.25)  # First quartile (25th percentile)
Q3 = df["Quantity"].quantile(0.75)  # Third quartile (75th percentile)
IQR = Q3 - Q1  # Interquartile range

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtering out outliers
df_out = df[(df["Quantity"] >= lower_bound) & (df["Quantity"] <= upper_bound)]

In [ ]:
# Creating a geopandas Dataframe after removing outliers
geometry = [Point(xy) for xy in zip(df_out["Longitude"], df_out["Latitude"])]
gdf_out = gpd.GeoDataFrame(df_out, geometry=geometry)

In [ ]:
# Scatter plot of sales without outliers
fig = px.scatter(
    gdf_out,
    x="Longitude",
    y="Latitude",
    color="Sales",
    hover_name="City",
    title="Scatter Plot of Locations by Latitude and Longitude",
)

# Adjust plot size
fig.update_layout(
    width=800, height=600  # Set the width of the plot  # Set the height of the plot
)
fig.show()
fig.write_html("Sales Scatterplot city hover.html")


In [ ]:
# Scatter plot with distinct colors for countries
fig = px.scatter(
    gdf_out,
    x="Longitude",
    y="Latitude",
    color="Country",  # Different colors for Germany and Poland
    size="Sales",  # Adjust point size based on Sales
    hover_name="City",
    title="Scatter Plot of Locations by Latitude and Longitude (Colored by Country)",
    color_discrete_map={
        "Germany": "red",  # Set Germany to red
        "Poland": "green",  # Set Poland to green
    },
)

# Adjust plot size
fig.update_layout(
    width=800, height=600  # Set the width of the plot  # Set the height of the plot
)

fig.show()
fig.write_html("Sales point.html")


In [ ]:
# Scatter plot
fig = px.scatter(
    gdf_out,
    x="Longitude",
    y="Latitude",
    color="Sales",
    hover_name="Distributor",
    title="Scatter Plot of Locations by Latitude and Longitude",
)

# Adjust plot size
fig.update_layout(
    width=800, height=600 
)
fig.show()
fig.write_html("Sales scatter distributor hover .html")


### Channel Analysis 

In [ ]:
# Let us analyze trend in sales for specific channels
sales_trends = gdf_out.groupby(['Channel', 'Year', 'Month'])['Sales'].sum().reset_index()

fig = px.line(
    sales_trends,
    x="Year",
    y="Sales",
    color="Channel",
    title="Monthly Sales Trends by Country"
)
fig.show()



In [ ]:
sales_seasonal = gdf_out.groupby(['Channel', 'Month'])['Sales'].mean().reset_index()

fig = px.line(
    sales_seasonal,
    x="Month",
    y="Sales",
    color="Channel",
    title="Average Monthly Sales Trends by Channel (Seasonality)"
)
fig.update_layout(
    xaxis=dict(tickmode="linear", tick0=1, dtick=1),  # Ensures all months appear
    xaxis_title="Month"
)
fig.show()
fig.write_html("Seasonality by channel.html")


In [ ]:
#Let us aggregate sales data by channel and subchannel
# Aggregate sales data
sales_by_channel = df.groupby(['Channel', 'Subchannel'])['Sales'].sum().reset_index()
sales_by_channel.sort_values(by='Sales', ascending=False, inplace=True)  


In [ ]:
#Plotting Barchart to show sales for each channel and subchannel
fig = px.bar(
    sales_by_channel,
    x='Channel',
    y='Sales',
    color='Subchannel',
    barmode='group',
    title="Total Sales by Channels and Subchannels",
    labels={'Sales': 'Total Sales'},
    text='Sales'
)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(
    xaxis_title="Channel",
    yaxis_title="Total Sales",
    legend_title="Subchannel",
    width=900,
    height=600
)
fig.show()
fig.write_html("Sales by subchannel barchart.html")


In [ ]:
#COMPARING SALES COUNT BY SUBCHANNEL FOR EACH COUNTRY
# Group data by Country and Subchannel to count sales entries
subchannel_sales = df.groupby(['Country', 'Subchannel']).size().reset_index(name='SalesEntries')

# Plot a grouped bar chart
fig = px.bar(
    subchannel_sales,
    x="Subchannel",  # Subchannel on x-axis
    y="SalesEntries",  # Number of sales entries on y-axis
    color="Country",  # Different colors for countries
    barmode="group",  # Grouped bars for comparison
    title="Sales Entries by Subchannel for Each Country",
    text_auto=True  # Display numbers on bars
)

# Adjust layout
fig.update_layout(
    xaxis_title="Subchannel",
    yaxis_title="Number of Sales Entries",
    legend_title="Country",
    width=900,
    height=600
)

fig.show()
fig.write_html("Salescount Subchannel groupbarchart.html")


* We can see that in Germany, the Government subchannel with 56,917 entries is doing better than the Private subchannel with 45,959 entries.
* While for Poland, the Private subchannel is doing better with 12,536 entries
* For Germany, the Retail subchannel of the Pharmacy channel has the overall highest entries while for Poland, the Private subchannel of the Hospital channel has the highest entry
* 


In [ ]:
#COMPARING TOTAL SALES BY SUBCHANNEL FOR EACH COUNTRY
# Group data by Country and Subchannel to calculate total sales
Channel_sales_total = df.groupby(['Country', 'Channel'])['Sales'].sum().reset_index()

# Plot a grouped bar chart
fig = px.bar(
    Channel_sales_total,
    x="Channel",  # Subchannel on x-axis
    y="Sales",  # Total sales on y-axis
    color="Country",  # Different colors for countries
    barmode="group",  # Grouped bars for comparison
    title="Total Sales by Channel for Each Country",
    text_auto=True  # Display numbers on bars
)

# Adjust layout
fig.update_layout(
    xaxis_title="Channel",
    yaxis_title="Total Sales",
    legend_title="Country",
    width=900,
    height=600
)

fig.show()
fig.write_html("Sales channel groupbarchart.html")
